In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class TimedElement():
    start_time: int
    end_time: int
    text: str

class CTMLine(TimedElement):
    pass

class RDAPI(TimedElement):
    pass

In [30]:
ctmlines = [
    CTMLine(0, 5, "foo"),
    CTMLine(5, 10, "foo"),
    CTMLine(10, 15, "foo"),
    CTMLine(21, 25, "foo"),
    CTMLine(30, 35, "foo"),
    CTMLine(37, 39, "foo"),
    CTMLine(41, 45, "foo"),
    CTMLine(50, 55, "foo"),
    CTMLine(61, 63, "foo"),
    CTMLine(60, 65, "foo"),
    CTMLine(70, 75, "foo"),
    CTMLine(80, 85, "foo"),
    CTMLine(90, 95, "foo"),
]

In [31]:
riksdag_output = [
    RDAPI(20, 35, "blah"),
    RDAPI(40, 55, "blah"),
    RDAPI(60, 75, "blah"),
]

In [5]:
from typing import List

@dataclass
class FilteredPair():
    ctmlines: List[CTMLine]
    riksdag_segments: List[RDAPI]
    speaker_name: str = ""

In [6]:
from copy import deepcopy

In [32]:
within = False
start = True

last_i = i = j = 0
pairs = []


while (i < len(ctmlines) - 1) and (j < len(riksdag_output)):
    if (start or not within) and ctmlines[i].end_time < riksdag_output[j].start_time:
        print("a", i, last_i, j)
        i += 1
    elif within and ctmlines[i].end_time <= riksdag_output[j].end_time:
        print("b", i, last_i, j)
        i += 1
    else:
        if within:
            rd = riksdag_output[j]
            spkr = riksdag_output[j].text
            within = False
            j += 1
            print("c", i, last_i, j)
        else:
            if start:
                start = False
            rd = None
            spkr = ""
            # check if we're not going straight into another within
            within = True
            print("d", i, last_i, j)
        pairs.append(FilteredPair(deepcopy(ctmlines[last_i:i]), rd, spkr))
        last_i = i
        i += 1
if i < len(ctmlines):
    pairs.append(FilteredPair(deepcopy(ctmlines[last_i:-1]), None, ""))


a 0 0 0
a 1 0 0
a 2 0 0
d 3 0 0
b 4 3 0
c 5 3 1
d 6 5 1
b 7 6 1
c 8 6 2
d 9 8 2
b 10 9 2
c 11 9 3


In [33]:
for pair in pairs:
    start_a = pair.ctmlines[0].start_time
    end_a = pair.ctmlines[-1].end_time
    start_b = pair.riksdag_segments.start_time if pair.riksdag_segments is not None else None
    end_b = pair.riksdag_segments.end_time if pair.riksdag_segments is not None else None
    print(start_a, end_a, start_b, end_b)

0 15 None None
21 35 20 35
37 39 None None
41 55 40 55
61 63 None None
60 75 60 75
80 85 None None
